# DATA ANALYSIS

In [9]:
import numpy as np
import pandas as pd
import datetime as dt

In [10]:
# Import and touch up ticker data
sp500 = pd.read_csv("Data/SP500.csv")
sp500.set_index('Unnamed: 0', inplace = True)
sp500.index.names = ['date']
sp500.set_index(pd.to_datetime(sp500.index), inplace = True)

sp500[sp500 == np.inf] = 0
sp500[sp500 == -np.inf] = 0

# Remove MMM because it has weird properties
sp500.drop('MMM', axis=1, inplace=True)

In [11]:
# Import and touch up indicator data
sp500_in = pd.read_csv("Data/SP500indicators.csv")
sp500_in.drop('Unnamed: 0', axis=1, inplace = True)
sp500_in.set_index(pd.to_datetime(sp500_in['calendardate']), inplace = True)
sp500_in.index.names = ['date']
sp500_in['year'] = sp500_in.index.year

In [12]:
sp500_in.head(10)

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,year
date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,MMM,MRY,2018-12-31,2018-12-31,2018-12-31,2019-04-26,-6.866000e+09,3.650000e+10,3.728200e+10,1.370900e+10,...,5.885000e+08,6.020000e+08,55.675,2.379200e+10,0.000000e+00,1637000000,243000000.0,40.971,6.465000e+09,2018
2017-12-31,MMM,MRY,2017-12-31,2017-12-31,2017-12-31,2019-04-26,-7.026000e+09,3.798700e+10,3.511825e+10,1.427700e+10,...,5.975000e+08,6.127000e+08,52.982,2.453800e+10,0.000000e+00,2679000000,310000000.0,41.150,6.590000e+09,2017
2016-12-31,MMM,MRY,2016-12-31,2016-12-31,2016-12-31,2019-04-26,-7.245000e+09,3.290600e+10,3.329350e+10,1.172600e+10,...,6.047000e+08,6.187000e+08,49.792,2.142000e+10,0.000000e+00,1995000000,299000000.0,35.688,5.507000e+09,2016
2015-12-31,MMM,MRY,2015-12-31,2015-12-31,2015-12-31,2019-04-26,-6.359000e+09,3.288300e+10,3.204300e+10,1.098600e+10,...,6.256000e+08,6.372000e+08,48.392,2.103300e+10,0.000000e+00,1982000000,332000000.0,34.245,3.868000e+09,2015
2014-12-31,MMM,MRY,2014-12-31,2014-12-31,2014-12-31,2019-04-26,-6.289000e+09,3.120900e+10,3.287550e+10,1.230300e+10,...,6.492000e+08,6.620000e+08,49.016,2.272400e+10,0.000000e+00,2028000000,435000000.0,35.612,6.339000e+09,2014
2013-12-31,MMM,MRY,2013-12-31,2013-12-31,2013-12-31,2019-04-26,-3.913000e+09,3.355000e+10,3.382825e+10,1.273300e+10,...,6.819000e+08,6.936000e+08,45.272,2.451700e+10,0.000000e+00,1841000000,417000000.0,36.680,5.235000e+09,2013
2012-12-31,MMM,MRY,2012-12-31,2012-12-31,2012-12-31,2019-04-26,-4.750000e+09,3.387600e+10,3.342725e+10,1.363000e+10,...,6.939000e+08,7.033000e+08,43.096,2.456600e+10,0.000000e+00,1840000000,371000000.0,35.526,7.430000e+09,2012
2011-12-31,MMM,MRY,2011-12-31,2011-12-31,2011-12-31,2019-04-26,-5.025000e+09,3.161600e+10,3.178550e+10,1.224000e+10,...,7.085000e+08,7.190000e+08,41.794,2.265300e+10,0.000000e+00,1674000000,355000000.0,32.301,6.799000e+09,2011
2018-12-31,ABT,MRY,2018-12-31,2018-12-31,2018-12-31,2019-05-01,-7.586000e+09,6.717300e+10,6.953325e+10,1.463200e+10,...,1.758000e+09,1.770000e+09,17.394,2.497700e+10,1.885000e+09,539000000,305000000.0,14.184,5.620000e+09,2018


In [13]:
# Cut down S&P 500 to stocks that existed for 8 years
sp500_cut = sp500.loc[pd.to_datetime('2011-12-30'):,:]
sp500_cut = sp500_cut.dropna(axis =1, how='any')

In [14]:
sp500_ar = sp500_cut.groupby(sp500_cut.index.year).sum().mean(axis=1)
sp500_ar = np.array(sp500_ar)
annual_rets = sp500_cut.groupby(sp500_cut.index.year).sum()
excess_rets = annual_rets.subtract(sp500_ar, axis=0)

excess_rets = excess_rets.reset_index()
excess_rets_melt = pd.melt(excess_rets, id_vars=['date'])
excess_rets_melt.rename(columns={'date':'year',
                                 'value':'excessreturns', 
                                 'variable':'ticker'}, 
                        inplace=True)

In [15]:
excess_rets_melt['year'] = excess_rets_melt['year'] - 1

In [16]:
sp500_prepped = pd.merge(sp500_in, excess_rets_melt, on = ['year','ticker'], how='left')
sp500_prepped = sp500_prepped[np.isfinite(sp500_prepped['excessreturns'])]
sp500_prepped = sp500_prepped.assign(beat = [1 if x > 0 else 0 for x in sp500_prepped['excessreturns']])

In [17]:
sp500_prepped.dropna(how='any', inplace=True)

In [18]:
sp500_prepped.columns.values

array(['ticker', 'dimension', 'calendardate', 'datekey', 'reportperiod',
       'lastupdated', 'accoci', 'assets', 'assetsavg', 'assetsc',
       'assetsnc', 'assetturnover', 'bvps', 'capex', 'cashneq',
       'cashnequsd', 'cor', 'consolinc', 'currentratio', 'de', 'debt',
       'debtc', 'debtnc', 'debtusd', 'deferredrev', 'depamor', 'deposits',
       'divyield', 'dps', 'ebit', 'ebitda', 'ebitdamargin', 'ebitdausd',
       'ebitusd', 'ebt', 'eps', 'epsdil', 'epsusd', 'equity', 'equityavg',
       'equityusd', 'ev', 'evebit', 'evebitda', 'fcf', 'fcfps', 'fxusd',
       'gp', 'grossmargin', 'intangibles', 'intexp', 'invcap',
       'invcapavg', 'inventory', 'investments', 'investmentsc',
       'investmentsnc', 'liabilities', 'liabilitiesc', 'liabilitiesnc',
       'marketcap', 'ncf', 'ncfbus', 'ncfcommon', 'ncfdebt', 'ncfdiv',
       'ncff', 'ncfi', 'ncfinv', 'ncfo', 'ncfx', 'netinc', 'netinccmn',
       'netinccmnusd', 'netincdis', 'netincnci', 'netmargin', 'opex',
       'opinc', 'p

In [19]:
excludes = ['ticker','dimension','calendardate', 'datekey', 
            'reportperiod','lastupdated','year','excessreturns']

In [20]:
X = sp500_prepped.drop(excludes, axis=1)
X.drop('beat', axis=1, inplace = True)

y = sp500_prepped['beat']

# Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13, shuffle=False)

## DECISION TREE

In [21]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV

In [45]:
tree_params ={'criterion':['gini','entropy'], 'max_depth': [2,3,4,5,6,7,8,9,10]}
clf = GridSearchCV(DecisionTreeClassifier(), tree_params, cv=5)

clf.fit(X_train, y_train)

print(clf.best_params_)

{'criterion': 'entropy', 'max_depth': 3}


In [46]:
dt_clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)
dt_clf.fit(X_train, y_train)
y_pred = dt_clf.predict(X_test)

from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

0.5343915343915344